In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline(model="ethan-cyj/Cross-Encoder-Finetuned")

c:\Users\USER\anaconda3\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pipe("answer: The primary function of the heart in the human body is to act as a muscular pump")

[{'label': 'LABEL_0', 'score': 0.0026125223375856876}]

In [3]:
inputs = tokenizer("heart muscles", return_tensors="pt")


NameError: name 'tokenizer' is not defined

In [25]:
import torch
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]


'LABEL_0'

In [2]:
#load from csv
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification

results = []
df = pd.read_csv('../../data/finetuning/test_dataset.csv')
tokenizer = AutoTokenizer.from_pretrained("./model")
model = AutoModelForSequenceClassification.from_pretrained("./model")
for index,row in df.iterrows():
    query = row['query']
    context = row['question']
    inputs = tokenizer(query, context, return_tensors='pt', truncation=True, padding=True)
    import torch
    with torch.no_grad():
        logits = model(**inputs).logits

    # Apply softmax to get probabilities
    probabilities = torch.softmax(logits, dim=-1)

    predicted_label = model.config.id2label[predicted_class_id]
    
    if predicted_label == 'LABEL_0':
        predicted_label = 0
    else: 
        predicted_label = 1

    # Get the predicted class and its probability
    predicted_class_id = probabilities.argmax().item()
    predicted_class_probability = probabilities[0, predicted_class_id].item()
    results.append({
        'query': query,
        'context': context,
        'actual_label': row['score'],
        'predicted_label': predicted_label
    })


results_df = pd.DataFrame(results)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


metrics = {
    'Base Accuracy': accuracy_score(results_df['actual_label'], results_df['predicted_label']),
    'Base Precision': precision_score(results_df['actual_label'], results_df['predicted_label']),
    'Base Recall': recall_score(results_df['actual_label'], results_df['predicted_label']),
    'Base F1': f1_score(results_df['actual_label'], results_df['predicted_label']),
}

print(metrics)


{'Base Accuracy': 0.5, 'Base Precision': 0.0, 'Base Recall': 0.0, 'Base F1': 0.0}


c:\Users\USER\anaconda3\envs\llm\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [5]:
df

,question_num,query,question,score
0,0,A nurse is assessing a patient who has been re...,For trained healthcare providers: - Checking o...,1.0
1,0,A nurse is assessing a patient who has been re...,Complications from Heimlich Manoeuvre If perfo...,0.0
2,1,A nurse is educating a patient with diabetes o...,1.7: THE CHAIN OF SURVIVAL The essential steps...,1.0
3,1,A nurse is educating a patient with diabetes o...,Assessing severity: 1. Mild: Partial Airway Ob...,0.0
4,2,A nurse is responding to a code blue in the em...,1.6: OTHER COMMON CAUSES OF CARDIAC ARRES...,1.0
...,...,...,...,...
159,79,What types of resources can be used to extingu...,3.1: The Importance of Early Defibrillation: A...,0.0
160,80,Which organs receive oxygen-rich blood directl...,The heart is a muscular pump located in the ce...,1.0
161,80,Which organs receive oxygen-rich blood directl...,3.9: CHILD/INFANT DEFIBRILLATION Use of Automa...,0.0
162,81,Which side of the heart receives oxygen-rich b...,"On an electrocardiogram (ECG), normal heart rh...",1.0


In [2]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd

train_dataset = pd.read_csv('../../data/finetuning/ce_finetuning_dataset.csv', header=0, names=['question_num', 'query', 'context', 'score'])

train_dataset['query'][0]


# Get the value of the first query
query_value = train_dataset['query'][0]

# Filter rows that contain the query_value in the 'query' column
filtered_dataset = train_dataset[train_dataset['query'] == query_value]

# Display the filtered dataset
filtered_dataset

,question_num,query,context,score
0,0,A nurse is assessing a patient who has been re...,1.8: SCDF MYRESPONDER MOBILE APPLICATION In Si...,1.0
1,0,A nurse is assessing a patient who has been re...,Recommendation for Automated External Defibril...,1.0
2,0,A nurse is assessing a patient who has been re...,4.1: CHILD CARDIO-PULMONARY RESUSCITATION (CPR...,1.0
3,0,A nurse is assessing a patient who has been re...,(D) Check for Danger Ensure that the rescuer ...,1.0
4,0,A nurse is assessing a patient who has been re...,(S) Shout to get help - If the child is unres...,1.0
...,...,...,...,...
74,0,A nurse is assessing a patient who has been re...,RETRIEVAL OF AUTOMATED EXTERNAL DEFIBRILLATOR:...,0.0
75,0,A nurse is assessing a patient who has been re...,The following are recommendations for retrieva...,0.0
76,0,A nurse is assessing a patient who has been re...,SAMPLE PRACTICAL SCENARIOS (FOR ADULTS): The ...,0.0
77,0,A nurse is assessing a patient who has been re...,5.4: HEIMLICH MAN OEUVRE Step 1: Assessment ...,0.0


In [3]:
import collections 
x = {}

for i in filtered_dataset['score'].tolist():
    if not i in x.keys():
        x[i] = 0
    x[i] += 1

x


filtered_dataset['query'][0]

'A nurse is assessing a patient who has been rescued from a drowning incident. What should the nurse do first to prevent cardiac arrest, and what other possible causes of cardiac arrest should the nurse consider?'

### Training Model

In [4]:
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, EarlyStoppingCallback
# Load the dataset


from torch.utils.data import Dataset, DataLoader

#train_dataset = pd.read_csv('../data/finetuning/ce_finetuning_dataset.csv', header=0, names=['question_num', 'query', 'context', 'score'])
train_dataset = filtered_dataset

class CrossEncoderFinetuningDatasetSample:
    def __init__(self, query, context, score):
        self.query = query
        self.context = context
        self.score = score

finetuning_dataset = [CrossEncoderFinetuningDatasetSample(d[1], d[2], d[3]) for d in train_dataset.values]
print(len(finetuning_dataset))

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, tokenizer, dataset):
        self.tokenizer = tokenizer
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]
        inputs = self.tokenizer(
            sample.query,
            sample.context,
            truncation=True,
            padding='max_length',
            max_length=512,
            return_tensors='pt'
        )
        inputs = {key: val.squeeze(0) for key, val in inputs.items()}
        inputs['labels'] = torch.tensor(sample.score, dtype=torch.float)
        return inputs
    

# Define a function to preprocess the dataset
def preprocess_function(examples):
    return tokenizer(examples['query'], examples['context'], truncation=True)

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("cross-encoder/ms-marco-MiniLM-L-12-v2")
model = AutoModelForSequenceClassification.from_pretrained("cross-encoder/ms-marco-MiniLM-L-12-v2", num_labels=1)

custom_dataset = CustomDataset(tokenizer, finetuning_dataset)
# Tokenize the dataset

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=20,  # Set a high number of epochs
    per_device_train_batch_size=10,
    save_steps=10_000,
    save_strategy="epoch",
    save_total_limit=2,
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    load_best_model_at_end=True,  # Load the best model at the end of training
)

# Initialize the Trainer with early stopping
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=custom_dataset,
    eval_dataset=custom_dataset,  # Use a separate validation set in practice
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],  # Stop if no improvement for 3 epochs
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained('./model')
tokenizer.save_pretrained('./model')


79


                                               
  5%|▌         | 8/160 [00:03<00:45,  3.31it/s]

{'eval_loss': 2.26896071434021, 'eval_runtime': 0.8417, 'eval_samples_per_second': 93.861, 'eval_steps_per_second': 11.881, 'epoch': 1.0}


                                                
 10%|█         | 16/160 [00:07<00:48,  2.98it/s]

{'eval_loss': 0.451978474855423, 'eval_runtime': 0.8348, 'eval_samples_per_second': 94.628, 'eval_steps_per_second': 11.978, 'epoch': 2.0}


...

                                                 
100%|██████████| 160/160 [01:32<00:00,  2.10it/s]

{'eval_loss': 0.009739531204104424, 'eval_runtime': 1.2193, 'eval_samples_per_second': 64.792, 'eval_steps_per_second': 8.201, 'epoch': 20.0}


100%|██████████| 160/160 [01:33<00:00,  1.71it/s]

{'train_runtime': 93.6306, 'train_samples_per_second': 16.875, 'train_steps_per_second': 1.709, 'train_loss': 0.4437899589538574, 'epoch': 20.0}


('./model\\tokenizer_config.json',
 './model\\special_tokens_map.json',
 './model\\vocab.txt',
 './model\\added_tokens.json',
 './model\\tokenizer.json')

### Testing Of Model (The model fking works omg!)

In [23]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, EarlyStoppingCallback
import torch

tokenizer = AutoTokenizer.from_pretrained("ethan-cyj/Cross-Encoder-Finetuned")
model = AutoModelForSequenceClassification.from_pretrained("ethan-cyj/Cross-Encoder-Finetuned", num_labels=1)

def get_score(query, user_answer, threshold = 0.5):
    # Tokenize the input data
    inputs = tokenizer(query, user_answer, return_tensors='pt', truncation=True, padding=True)
    
    # Make predictions
    with torch.no_grad():
        logits = model(**inputs).logits

    # Apply sigmoid to get probabilities
    probabilities = torch.sigmoid(logits)
    predicted_class = 0
    if probabilities > threshold:
        return 1 
    else: 
        return predicted_class

test_df = pd.read_csv('../../data/finetuning/test_dataset.csv')

# Example usage
query = test_df['query'][1]
user_answer = test_df['question'][1]
print(get_score(query,user_answer))

0


In [20]:
test_df

,question_num,query,question,score
0,0,A nurse is assessing a patient who has been re...,For trained healthcare providers: - Checking o...,1.0
1,0,A nurse is assessing a patient who has been re...,Complications from Heimlich Manoeuvre If perfo...,0.0
2,1,A nurse is educating a patient with diabetes o...,1.7: THE CHAIN OF SURVIVAL The essential steps...,1.0
3,1,A nurse is educating a patient with diabetes o...,Assessing severity: 1. Mild: Partial Airway Ob...,0.0
4,2,A nurse is responding to a code blue in the em...,1.6: OTHER COMMON CAUSES OF CARDIAC ARRES...,1.0
...,...,...,...,...
159,79,What types of resources can be used to extingu...,3.1: The Importance of Early Defibrillation: A...,0.0
160,80,Which organs receive oxygen-rich blood directl...,The heart is a muscular pump located in the ce...,1.0
161,80,Which organs receive oxygen-rich blood directl...,3.9: CHILD/INFANT DEFIBRILLATION Use of Automa...,0.0
162,81,Which side of the heart receives oxygen-rich b...,"On an electrocardiogram (ECG), normal heart rh...",1.0


In [1]:
from sentence_transformers import CrossEncoder

# 1. Load the pre-trained model
model = CrossEncoder('./model')

# 2. Define your question and potential answers 
question = train_dataset['query'][0]
potential_answers = [
    train_dataset['context'][1],
    train_dataset['context'][len(train_dataset) - 1],
    train_dataset['context'][len(train_dataset) - 2],
]

# 3. Get relevance scores for each (question, answer) pair
scores = model.predict([(question, answer) for answer in potential_answers])

# 4. Find the index of the answer with the highest score
best_answer_index = scores.argmax()

# Print the results
print("Question:", question)
print("Best Answer:", potential_answers[best_answer_index])

c:\Users\USER\anaconda3\envs\hack\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


KeyboardInterrupt: 

In [12]:
train_dataset['context'][len(train_dataset) - 1]

'(C) Chest Compressions   - Site of chest compression should be at the centre of the chest at the lower half of the sternum (breastbone). Ensure adequate exposure of the chest.  Locate the correct hand position for chest compressions:       - Use your index finger to d raw an imaginary line      between the nipples  to the centre of the sternum.        - Place your middle (third), and ring (fourth) fingers next      to your index finger on the center of      the infant’s chest.            - Lift up your index finger but maintain the middle and      ring fingers on the sternum and commence chest      compressions  using the pulp of the fingers.        - Depth of chest compression for infants must be 1/3 the anterior-posterior diameter of the infant’s chest (about 3 to 4 cm).      - The compression rate is 100–120 per minute.       - Ensure complete recoil of the chest wall after each compression. '

In [13]:
filtered_dataset

,question_num,query,context,score
0,0,A nurse is assessing a patient who has been re...,1.8: SCDF MYRESPONDER MOBILE APPLICATION In Si...,1.0
1,0,A nurse is assessing a patient who has been re...,Recommendation for Automated External Defibril...,1.0
2,0,A nurse is assessing a patient who has been re...,4.1: CHILD CARDIO-PULMONARY RESUSCITATION (CPR...,1.0
3,0,A nurse is assessing a patient who has been re...,(D) Check for Danger Ensure that the rescuer ...,1.0
4,0,A nurse is assessing a patient who has been re...,(S) Shout to get help - If the child is unres...,1.0
...,...,...,...,...
74,0,A nurse is assessing a patient who has been re...,RETRIEVAL OF AUTOMATED EXTERNAL DEFIBRILLATOR:...,0.0
75,0,A nurse is assessing a patient who has been re...,The following are recommendations for retrieva...,0.0
76,0,A nurse is assessing a patient who has been re...,SAMPLE PRACTICAL SCENARIOS (FOR ADULTS): The ...,0.0
77,0,A nurse is assessing a patient who has been re...,5.4: HEIMLICH MAN OEUVRE Step 1: Assessment ...,0.0
